In [4]:
import sys
sys.path.append("../..")

import dataInterpreter as dt
import enrichmentAnalysis as ea
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
pd.options.mode.chained_assignment = None  # default='warn'

path = "C:\\Users\\Pedro\\Desktop\\bicpams_5.1\\data\\latecovid\\"
results_path = "C:\\Users\\Pedro\\Documents\\MasterThesis\\Biclustering\\Results\\"

In [5]:
filtered_data_01 = pd.read_csv(path + 'data-p01.csv', index_col = 0, sep = '\t')
filtered_data_02 = pd.read_csv(path + 'data-p02.csv', index_col = 0, sep = '\t')
filtered_data_02_03 = pd.read_csv(path + 'data-p02-03.csv', index_col = 0, sep = '\t')
filtered_data_05 = pd.read_csv(path + 'data-p05.csv', index_col = 0, sep = '\t')

top200 = pd.read_csv(path + 'data-top200.csv', index_col = 0, sep = '\t')
top500 = pd.read_csv(path + 'data-top500.csv', index_col = 0, sep = '\t')
top1000 = pd.read_csv(path + 'data-top1000.csv', index_col = 0, sep = '\t')
top2000 = pd.read_csv(path + 'data-top2000.csv', index_col = 0, sep = '\t')
top5000 = pd.read_csv(path + 'data-top5000.csv', index_col = 0, sep = '\t')

In [50]:
baseline_filtered_data_01 = ea.getEnrichment(list(filtered_data_01.index), 'GO_Biological_Process_2018')
#baseline_filtered_data_02 = ea.getEnrichment(list(filtered_data_02.index), 'GO_Biological_Process_2018')
#baseline_filtered_data_05 = ea.getEnrichment(list(filtered_data_05.index), 'GO_Biological_Process_2018')

#baseline_top200 = ea.getEnrichment(list(top200.index), 'GO_Biological_Process_2018')
#baseline_top500 = ea.getEnrichment(list(top500.index), 'GO_Biological_Process_2018')
#baseline_top1000 = ea.getEnrichment(list(top1000.index), 'GO_Biological_Process_2018')
#baseline_top2000 = ea.getEnrichment(list(top2000.index), 'GO_Biological_Process_2018')
#baseline_top5000 = ea.getEnrichment(list(top5000.index), 'GO_Biological_Process_2018')

Order of returned results is: Rank, Term name, P-value, Z-score, Combined score, Overlapping genes, Adjusted p-value, Old p-value, Old adjusted p-value


In [124]:
def binary_search_results(data, value, data_selector_lambda = lambda x, i: x[i]):
    start = 0
    end = len(data) - 1
    
    # detect a boundary in an ordered vector
    while start < end:
        i = int((end + start) / 2)
        
        if data_selector_lambda(data, i) < value and data_selector_lambda(data, i + 1) >= value:
            break
        
        if data_selector_lambda(data, i) >= value:
            # go left
            end = i - 1
        else:
            # go right
            start = i + 1
            
    i = int((end + start) / 2)
    return i + 1

11

In [ ]:
binary_search_results(ea.getEnrichment(list(filtered_data_01.index), 'GO_Biological_Process_2018'), 0.01, lambda x, i: x[i][6])

In [ ]:
## print('p01 ->', len([x for x in baseline_filtered_data_01['GO_Biological_Process_2018'] if x[6] < 0.01]))
print('p02 ->', len([x for x in baseline_filtered_data_02['GO_Biological_Process_2018'] if x[6] < 0.01]))
print('p05 ->', len([x for x in baseline_filtered_data_05['GO_Biological_Process_2018'] if x[6] < 0.01]))

print('\nANOVA 200 ->', len([x for x in baseline_top200['GO_Biological_Process_2018'] if x[6] < 0.01]))
print('ANOVA 500 ->', len([x for x in baseline_top500['GO_Biological_Process_2018'] if x[6] < 0.01]))
print('ANOVA 1000 ->', len([x for x in baseline_top1000['GO_Biological_Process_2018'] if x[6] < 0.01]))
print('ANOVA 2000 ->', len([x for x in baseline_top2000['GO_Biological_Process_2018'] if x[6] < 0.01]))
print('ANOVA 5000 ->', len([x for x in baseline_top5000['GO_Biological_Process_2018'] if x[6] < 0.01]))

In [115]:
import re

def import_bicPAM(filename):
    file = open(filename, 'r')
    
    # get number of biclusters found
    num_biclusters = int(re.search("[0-9]+", file.readline()).group())
    
    rx = re.compile('[-+]?(\d+(\.\d*)?|\.\d+)([eE][-+]?\d+)?', re.VERBOSE)
    
    # read summary lines
    for i in range(num_biclusters):
        bic_info = file.readline() #.split(' ')
        #print(bic_info[3][3:-1].split(','))
    
    file.readline()
    
    result = []
    
    for i in range(num_biclusters):
        bic_info = file.readline().split(' ')
        size = eval(bic_info[2])
        conditions = bic_info[3][3:-1].split(',')
        pvalue = float(bic_info[6].split('=')[1])
        
        genes = []
        for j in range(size[1]):
            genes.append(file.readline().split('\t')[0])
            
        enriched_terms = ea.getEnrichment(genes, 'GO_Biological_Process_2018')['GO_Biological_Process_2018']
        
        bic = {'size': size, 'conditions': conditions, 'genes': genes, 'pvalue': pvalue, 'enriched_terms': enriched_terms}
        result.append(bic)
        
        file.readline()
        file.readline()
    
    return result
    

In [116]:
def import_bics_biclust(filename):
    file = open(filename, 'r')
    
    # Read line of algorithm parameters
    line = file.readline()
    
    result = []
    
    while line != '':
        # size line, gene line, conditions
        for i in range(3):
            size_text = file.readline().split(' ')
            if len(size_text) != 2: 
                return result
            size = (int(size_text[1]), int(size_text[0]))
            
            genes = file.readline()[:-1].split(' ')
            conditions = file.readline()[:-1].split(' ')
            
            enriched_terms = ea.getEnrichment(list(genes), 'GO_Biological_Process_2018')['GO_Biological_Process_2018']
            
            bic = {'size': size, 'conditions': conditions, 'genes': genes, 'enriched_terms': enriched_terms}
            result.append(bic)
    
    # usually shouldn't reach this
    return result

In [21]:
bicpam_p01[0]['size']

(7, 272)

In [117]:
results = {}

results['bicpam_p01'] = import_bicPAM(results_path + 'result-p01-bicPAM.txt')
results['bicpam_p02'] = import_bicPAM(results_path + 'result-p02-bicPAM.txt')
results['bicpam_p05'] = import_bicPAM(results_path + 'result-p05-bicPAM.txt')
results['bicpam_anova200'] = import_bicPAM(results_path + 'result-top200-bicPAM.txt')
results['bicpam_anova500'] = import_bicPAM(results_path + 'result-top500-bicPAM.txt')
results['bicpam_anova1000'] = import_bicPAM(results_path + 'result-top1000-bicPAM.txt')
results['bicpam_anova2000'] = import_bicPAM(results_path + 'result-top2000-bicPAM.txt')
results['bicpam_anova5000'] = import_bicPAM(results_path + 'result-top5000-bicPAM.txt')

results['cheng_church_p01'] = import_bics_biclust(results_path + 'results-p01-chengAndChurch.txt')
results['cheng_church_p02'] = import_bics_biclust(results_path + 'results-p02-chengAndChurch.txt')
results['cheng_church_p05'] = import_bics_biclust(results_path + 'results-p05-chengAndChurch.txt')
results['cheng_church_anova200'] = import_bics_biclust(results_path + 'results-top200-chengAndChurch.txt')
results['cheng_church_anova500'] = import_bics_biclust(results_path + 'results-top500-chengAndChurch.txt')
results['cheng_church_anova1000'] = import_bics_biclust(results_path + 'results-top1000-chengAndChurch.txt')
results['cheng_church_anova2000'] = import_bics_biclust(results_path + 'results-top2000-chengAndChurch.txt')
results['cheng_church_anova5000'] = import_bics_biclust(results_path + 'results-top5000-chengAndChurch.txt')

results['plaid_p01'] = import_bics_biclust(results_path + 'results-p01-plaid.txt')
results['plaid_p02'] = import_bics_biclust(results_path + 'results-p02-plaid.txt')
results['plaid_p05'] = import_bics_biclust(results_path + 'results-p05-plaid.txt')
results['plaid_anova200'] = import_bics_biclust(results_path + 'results-top200-plaid.txt')
results['plaid_anova500'] = import_bics_biclust(results_path + 'results-top500-plaid.txt')
results['plaid_anova1000'] = import_bics_biclust(results_path + 'results-top1000-plaid.txt')
results['plaid_anova2000'] = import_bics_biclust(results_path + 'results-top2000-plaid.txt')
results['plaid_anova5000'] = import_bics_biclust(results_path + 'results-top5000-plaid.txt')

results['xMotifs_p01'] = import_bics_biclust(results_path + 'results-p01-xMotifs.txt')
results['xMotifs_p02'] = import_bics_biclust(results_path + 'results-p02-xMotifs.txt')
results['xMotifs_p05'] = import_bics_biclust(results_path + 'results-p05-xMotifs.txt')
results['xMotifs_anova200'] = import_bics_biclust(results_path + 'results-top200-xMotifs.txt')
results['xMotifs_anova500'] = import_bics_biclust(results_path + 'results-top500-xMotifs.txt')
results['xMotifs_anova1000'] = import_bics_biclust(results_path + 'results-top1000-xMotifs.txt')
results['xMotifs_anova2000'] = import_bics_biclust(results_path + 'results-top2000-xMotifs.txt')
results['xMotifs_anova5000'] = import_bics_biclust(results_path + 'results-top5000-xMotifs.txt')

Order of returned results is: Rank, Term name, P-value, Z-score, Combined score, Overlapping genes, Adjusted p-value, Old p-value, Old adjusted p-value
Order of returned results is: Rank, Term name, P-value, Z-score, Combined score, Overlapping genes, Adjusted p-value, Old p-value, Old adjusted p-value
Order of returned results is: Rank, Term name, P-value, Z-score, Combined score, Overlapping genes, Adjusted p-value, Old p-value, Old adjusted p-value
Order of returned results is: Rank, Term name, P-value, Z-score, Combined score, Overlapping genes, Adjusted p-value, Old p-value, Old adjusted p-value
Order of returned results is: Rank, Term name, P-value, Z-score, Combined score, Overlapping genes, Adjusted p-value, Old p-value, Old adjusted p-value
Order of returned results is: Rank, Term name, P-value, Z-score, Combined score, Overlapping genes, Adjusted p-value, Old p-value, Old adjusted p-value
Order of returned results is: Rank, Term name, P-value, Z-score, Combined score, Overlap

In [118]:
import json

with open('processed_results.txt', 'w') as file:
     file.write(json.dumps(results)) # use `json.loads` to do the reverse

NameError: name 'json' is not defined

In [135]:
def prettify_biclust_method(method):
    if method == 'bicpam':
        return 'BicPAM'
    elif method == 'cheng_church':
        return 'Cheng and Church'
    elif method == 'plaid':
        return 'Plaid'
    else:
        return method

def prettify_gene_select_method(method):
    if method.startswith('p'):
        return 'p-value < 0.' + method.rpartition('p')[-1] + ' (Mann-Whitney U test)'
    elif method.startswith('anova'):
        return 'ANOVA, top ' + method.rpartition('anova')[-1] + ' features'

In [153]:
table1 = {'Number of Biclusters': [], 'Average Number of Genes per Bicluster': [], 'Standard Deviation of Genes per Bicluster': [], 'Average Number of Conditions per Bicluster': [], 'Standard Deviation of Conditions per Bicluster': [], 'Average of Enriched Terms per Bicluster': []}
table1_index = []

for method in results:
    method_info = method.rpartition('_')
    
    table1_index.append((prettify_biclust_method(method_info[0]), prettify_gene_select_method(method_info[2])))
    table1['Number of Biclusters'].append(len(results[method]))
    table1['Average Number of Genes per Bicluster'].append(np.mean([x['size'][1] for x in results[method]]))
    table1['Standard Deviation of Genes per Bicluster'].append(np.std([x['size'][1] for x in results[method]]))
    table1['Average Number of Conditions per Bicluster'].append(np.mean([x['size'][0] for x in results[method]]))
    table1['Standard Deviation of Conditions per Bicluster'].append(np.std([x['size'][0] for x in results[method]]))
    table1['Average of Enriched Terms per Bicluster'].append(np.mean([binary_search_results(x['enriched_terms'], 0.01, lambda x, i: x[i][6]) for x in results[method]]))

In [154]:
pd.DataFrame(table1, pd.MultiIndex.from_tuples(table1_index))

Number of Biclusters  \
BicPAM           p-value < 0.01 (Mann-Whitney U test)                    80   
                 p-value < 0.02 (Mann-Whitney U test)                    77   
                 p-value < 0.05 (Mann-Whitney U test)                    79   
                 ANOVA, top 200 features                                  7   
                 ANOVA, top 500 features                                 47   
                 ANOVA, top 1000 features                                20   
                 ANOVA, top 2000 features                                98   
                 ANOVA, top 5000 features                                57   
Cheng and Church p-value < 0.01 (Mann-Whitney U test)                    50   
                 p-value < 0.02 (Mann-Whitney U test)                   100   
                 p-value < 0.05 (Mann-Whitney U test)                   100   
                 ANOVA, top 200 features                                  8   
                 ANOVA, top 500 features                                 28   
                 ANOVA, top 1000 features                                56   
                 ANOVA, top 2000 features                               100   
                 ANOVA, top 5000 features                               100   
Plaid            p-value < 0.01 (Mann-Whitney U test)                    10   
                 p-value < 0.02 (Mann-Whitney U test)                    10   
                 p-value < 0.05 (Mann-Whitney U test)                    10   
                 ANOVA, top 200 features                                  8   
                 ANOVA, top 500 features                                 10   
                 ANOVA, top 1000 features                                10   
                 ANOVA, top 2000 features                                10   
                 ANOVA, top 5000 features                                10   
xMotifs          p-value < 0.01 (Mann-Whitney U test)                    10   
                 p-value < 0.02 (Mann-Whitney U test)                    10   
                 p-value < 0.05 (Mann-Whitney U test)                    10   
                 ANOVA, top 200 features                                  6   
                 ANOVA, top 500 features                                 10   
                 ANOVA, top 1000 features                                10   
                 ANOVA, top 2000 features                                10   
                 ANOVA, top 5000 features                                10   

                                                       Average Number of Genes per Bicluster  \
BicPAM           p-value < 0.01 (Mann-Whitney U test)                             208.025000   
                 p-value < 0.02 (Mann-Whitney U test)                             922.701299   
                 p-value < 0.05 (Mann-Whitney U test)                            3526.658228   
                 ANOVA, top 200 features                                          188.285714   
                 ANOVA, top 500 features                                          380.191489   
                 ANOVA, top 1000 features                                         676.050000   
                 ANOVA, top 2000 features                                         943.867347   
                 ANOVA, top 5000 features                                        2106.175439   
Cheng and Church p-value < 0.01 (Mann-Whitney U test)                              15.600000   
                 p-value < 0.02 (Mann-Whitney U test)                              23.870000   
                 p-value < 0.05 (Mann-Whitney U test)                              55.900000   
                 ANOVA, top 200 features                                           25.000000   
                 ANOVA, top 500 features                                           17.821429   
                 ANOVA, top 1000 features                                          17.857143   
       

In [147]:
results['plaid_p01'][2]

{'size': (18, 94),
 'conditions': ['Series1_NHBE_Mock_1',
  'Series1_NHBE_Mock_2',
  'Series1_NHBE_Mock_3',
  'Series9_NHBE_Mock_1',
  'Series9_NHBE_Mock_2',
  'Series9_NHBE_Mock_3',
  'Series9_NHBE_Mock_4',
  'Series1_NHBE_SARS.CoV.2_1',
  'Series1_NHBE_SARS.CoV.2_2',
  'Series1_NHBE_SARS.CoV.2_3',
  'Series9_NHBE_IAV_1',
  'Series9_NHBE_IAV_2',
  'Series9_NHBE_IAV_3',
  'Series9_NHBE_IAV_4',
  'Series9_NHBE_IAVdNS1_1',
  'Series9_NHBE_IAVdNS1_2',
  'Series9_NHBE_IAVdNS1_3',
  'Series9_NHBE_IAVdNS1_4'],
 'genes': ['LOC100506801',
  'LUZP1',
  'INPP5B',
  'PTPRF',
  'ROR1',
  'CTSS',
  'PFKP',
  'HKDC1',
  'GRID1',
  'HSPA12A',
  'SYT9',
  'DENND5A',
  'NUCB2',
  'RASGRP2',
  'UBASH3B',
  'DUSP16',
  'GPR19',
  'GRIN2B',
  'AMIGO2',
  'C12orf65',
  'STX2',
  'DCLK1',
  'LCP1',
  'HAUS4',
  'AJUBA',
  'SNAPC1',
  'KLC1',
  'UACA',
  'HCN4',
  'RGS11',
  'FOXL1',
  'PITPNA-AS1',
  'TRIM16L',
  'DHRS13',
  'ITGA3',
  'C1QTNF1',
  'PRTN3',
  'CELF5',
  'LINC00662',
  'CD22',
  'ZNF566',
  

In [158]:
results['plaid_p01'][2]['enriched_terms']

{'GO_Biological_Process_2018': [[1,
   'regulation of signal transduction (GO:0009966)',
   0.00011528014909489325,
   7.006408300274641,
   63.535128588717825,
   ['UBASH3B', 'ITGA3', 'AXL', 'HHIP', 'ROR1', 'FOXL1', 'RGS11'],
   0.06985977035150531,
   0,
   0],
  [2,
   'negative regulation of cell communication (GO:0010648)',
   0.0005003052558027592,
   11.911111111111111,
   90.52792408763345,
   ['UBASH3B', 'AXL', 'HHIP', 'ROR1'],
   0.11251701215078214,
   0,
   0],
  [3,
   'negative regulation of signaling (GO:0023057)',
   0.0006331080518577728,
   11.154430379746836,
   82.15092356215999,
   ['UBASH3B', 'AXL', 'HHIP', 'ROR1'],
   0.11251701215078214,
   0,
   0],
  [4,
   'extracellular matrix organization (GO:0030198)',
   0.0007426865488500471,
   6.01803913575214,
   43.361395086500494,
   ['ITGA3', 'COL4A6', 'COL4A5', 'LCP1', 'CTSS', 'MATN3'],
   0.11251701215078214,
   0,
   0],
  [5,
   'positive regulation of cation transmembrane transport (GO:1904064)',
   0.00283885

In [148]:
for gene in results['plaid_p01'][2]['genes']:
    print(gene)

LOC100506801
LUZP1
INPP5B
PTPRF
ROR1
CTSS
PFKP
HKDC1
GRID1
HSPA12A
SYT9
DENND5A
NUCB2
RASGRP2
UBASH3B
DUSP16
GPR19
GRIN2B
AMIGO2
C12orf65
STX2
DCLK1
LCP1
HAUS4
AJUBA
SNAPC1
KLC1
UACA
HCN4
RGS11
FOXL1
PITPNA-AS1
TRIM16L
DHRS13
ITGA3
C1QTNF1
PRTN3
CELF5
LINC00662
CD22
ZNF566
AXL
HSD17B14
ZNF160
PRKCG
PTPRH
ZFP28
IAH1
CYS1
MATN3
FAM161A
ST3GAL5
LIPT1
NPAS2
C2orf49
NAB1
TRIB3
EEF1A2
IFNAR2
IL10RB
STAC
SLC35G2
SHOX2
LIPH
CXCL5
HHIP
MAP1B
SLCO4C1
EFNA5
LOC100268168
STC2
TPBG
AKAP12
ZNF853
BMPER
LINC00525
SEMA3C
GATAD1
TMEM130
LOC254896
NEFL
PNMA2
ASPH
MYBL1
ZNF34
PLIN2
SVEP1
PPP1R26
CRLF2
CLCN4
GSPT2
COL4A6
COL4A5
NLGN4Y


In [167]:
terms = {}

for method in results:
    for biclust in results[method]: 
        biclust_terms = biclust['enriched_terms']
        if 'GO_Biological_Process_2018' in biclust['enriched_terms']:
            biclust_terms = biclust['enriched_terms']['GO_Biological_Process_2018']
        
        for term in biclust_terms:
            if term[6] >= 0.01:
                break

            if term[1] not in terms:
                terms[term[1]] = {'p-value aggregate score': term[6], 'Total occurrences': 1}
            else:
                terms[term[1]]['p-value aggregate score'] = term[6]
                terms[term[1]]['Total occurrences'] += 1

for term in terms:
    terms[term]['p-value aggregate score'] /= terms[term]['Total occurrences']

In [173]:
pd.DataFrame(terms).T.nlargest(50, 'Total occurrences')

,p-value aggregate score,Total occurrences
cytokine-mediated signaling pathway (GO:0019221),5.539747e-08,307.0
cellular response to interferon-gamma (GO:0071346),2.553413e-12,294.0
interferon-gamma-mediated signaling pathway (GO:0060333),6.802809e-08,250.0
chemokine-mediated signaling pathway (GO:0070098),2.181056e-07,243.0
regulation of immune response (GO:0050776),1.385504e-05,240.0
response to interferon-gamma (GO:0034341),7.237031e-08,235.0
cellular response to cytokine stimulus (GO:0071345),1.953257e-06,232.0
epidermis development (GO:0008544),1.452056e-05,229.0
positive regulation of cytosolic calcium ion concentration (GO:0007204),1.133431e-06,225.0
regulation of T cell proliferation (GO:0042129),2.096474e-05,223.0
